<a id=top></a>

# **目次**

<b>
    <details>
        <summary>
            <a href="#modules", style="font-size: xx-large">1. モジュールインポート</a>
            <ul>※サードパーティライブラリ>>>自作モジュール>>>（ここまで本ipynb外）>>>自作関数（本ipynb内）</ul>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#data", style="font-size: xx-large">2. オリジナルデータインポート</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#patentcount", style="font-size: xx-large">3. 特許数</a>
        </summary>
        <table></table>
    </details>
    <details>
        <summary>
            <a href="#calculateindicator", style="font-size: xx-large">4. 各指標</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#output", style="font-size: xx-large">5. ファイルに出力</a>
        </summary>
    </details>
</b>


---


<a id=modules></a>

## **1. モジュールインポート**


In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [ ]:
import initial_condition


In [ ]:
global data_dir, output_dir
data_dir = '../../data/interim/internal/filtered_after_agg/'
output_dir = '../../data/processed/internal/'
ex_dir = '../../data/processed/external/schmoch/'


In [ ]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM
# top_p_or_num = ('p', 100)
region_corporation = initial_condition.REGION_CORPORATION
# region_corporation = 'right_person_addr'
applicant_weight = initial_condition.APPLICANT_WEIGHT

classification = initial_condition.CLASSIFICATION
class_weight = initial_condition.CLASS_WEIGHT

input_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'
output_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'


In [ ]:
input_condition

In [ ]:
def kh_ki(c_df, classification, n=19):
    kh1_ki1_df = pd.merge(c_df.copy(), 
                        c_df[c_df['mcp']==1].groupby([region_corporation])[['ubiquity']].sum().reset_index(drop=False).copy().rename(columns={'ubiquity':'kh_1'}), 
                        on=[region_corporation], how='left')
    kh1_ki1_df = pd.merge(kh1_ki1_df.copy(), 
                        c_df[c_df['mcp']==1].groupby([classification])[['diversity']].sum().reset_index(drop=False).copy().rename(columns={'diversity':'ki_1'}), 
                        on=[classification], how='left')
    kh1_ki1_df['kh_1'] = kh1_ki1_df['kh_1'] / kh1_ki1_df['diversity']
    kh1_ki1_df['ki_1'] = kh1_ki1_df['ki_1'] / kh1_ki1_df['ubiquity']
    kh_ki_df = kh1_ki1_df.copy()
    for i in range(n):
        kh_ki_df = pd.merge(kh_ki_df, 
                            kh_ki_df[kh_ki_df['mcp']==1].groupby([region_corporation])[[f'ki_{i+1}']].sum().reset_index(drop=False).copy()\
                                        .rename(columns={f'ki_{i+1}':f'kh_{i+2}'}), 
                            on=[region_corporation], how='left')
        kh_ki_df = pd.merge(kh_ki_df, 
                            kh_ki_df[kh_ki_df['mcp']==1].groupby([classification])[[f'kh_{i+1}']].sum().reset_index(drop=False).copy()\
                                        .rename(columns={f'kh_{i+1}':f'ki_{i+2}'}), 
                            on=[classification], how='left')
        kh_ki_df[f'kh_{i+2}'] = kh_ki_df[f'kh_{i+2}'] / kh_ki_df['diversity']
        kh_ki_df[f'ki_{i+2}'] = kh_ki_df[f'ki_{i+2}'] / kh_ki_df['ubiquity']
    return kh_ki_df


---


<a id=data></a>

## **2. オリジナルデータインポート**


In [ ]:
schmoch_df = pd.read_csv(f'{ex_dir}35.csv', 
                         encoding='utf-8', 
                         sep=',', 
                         usecols=['Field_number', 'Field_en']
                         ).drop_duplicates()

In [ ]:
reg_num_top_df = pd.read_csv(f'{data_dir}{input_condition}.csv', 
                             encoding='utf-8',
                             sep=',')
reg_num_top_df

In [ ]:
reg_num_top_df[region_corporation].nunique()

<a href=#top>先頭に戻る</a>

---


<a id=calculateindicator></a>

## **4. 各指標**


In [ ]:
trade_cols = {'time':f'{ar}_{year_style}_period', 'loc':region_corporation, 'prod':classification, 'val':'reg_num'}
rename_col_dict = {'eci':'kci', 'pci':'tci'}
col_order_list = [f'{ar}_{year_style}_period', region_corporation, classification, 'reg_num', 'rca', 'mcp', 'diversity', 'ubiquity', 'kci', 'tci']


In [ ]:
c_df = ecomplexity(reg_num_top_df,
                   cols_input = trade_cols, 
                   rca_mcp_threshold = 1)
# c_out_df = c_df.copy()
c_df = c_df[c_df['reg_num'] > 0]\
           .rename(columns=rename_col_dict)\
           [col_order_list]
c_df = pd.concat([kh_ki(c_df[c_df[f'{ar}_{year_style}_period'] == period], classification) for period in c_df[f'{ar}_{year_style}_period'].unique()], 
                 axis='index', 
                 ignore_index=True)

# for segment in c_df[f'{ar}_{year_style}_period'].unique():
#     display(c_df[c_df[f'{ar}_{year_style}_period'] == segment].head())
#     display(c_df[c_df[f'{ar}_{year_style}_period'] == segment].describe())
#     print(c_df[c_df[f'{ar}_{year_style}_period'] == segment].info())
#     print('\n')


In [ ]:
c_df.sort_values(by=[f'{ar}_{year_style}_period', 'kci'], ascending=[True, False])

In [ ]:
c_df[(c_df[classification]==22)&(c_df[f'{ar}_{year_style}_period']=='1981-1990')]

<a href="#top">先頭に戻る</a>

---


<a id=output></a>

## **5. ファイルに出力**

<a id=rightperson></a>

### **5.1. 特許権者**


In [ ]:

right_person_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', region_corporation])[['reg_num']].sum().reset_index(drop=False), 
                           c_df.groupby([f'{ar}_{year_style}_period', region_corporation])[[classification]].nunique().reset_index(drop=False), 
                           on=[f'{ar}_{year_style}_period', region_corporation], 
                           how='inner')
right_person_df = pd.merge(right_person_df, 
                           c_df[[f'{ar}_{year_style}_period', region_corporation, 'diversity', 'kci']\
                               +[f'kh_{i}' for i in range(1, 20+1)]]\
                               .drop_duplicates(keep='first'), 
                           on=[f'{ar}_{year_style}_period', region_corporation], 
                           how='inner')
# for period in right_person_df[f'{ar}_{year_style}_period'].unique():
#     right_person_df

# for period in right_person_df[f'{ar}_{year_style}_period'].unique():
#     for i in range(1, 20+1):
#         value = right_person_df[right_person_df[f'{ar}_{year_style}_period']==period]
#         right_person_df[right_person_df[f'{ar}_{year_style}_period']==period][f'kh_{i}'] = (value[f'kh_{i}'] - value[f'kh_{i}'].mean()) / value[f'kh_{i}'].std()
#     display(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].head())
#     display(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].describe())
#     print(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].info())
#     print('\n')
# right_person_df['reg_num'] = right_person_df['reg_num'].astype(np.int64)

In [ ]:
schmoch_df

In [ ]:
sample = pd.merge(c_df, schmoch_df, 
         left_on=classification, right_on='Field_number', how='left')
sample[(sample[f'{ar}_{year_style}_period']==f'{year_start}-{year_end}')&(sample['Field_en']=='Digital communication')&\
    (sample['mcp']==1)]

In [ ]:
right_person_df[right_person_df[f'{ar}_{year_style}_period']==f'{year_start}-{year_end}'].sort_values('kci', ascending=False)

In [ ]:
right_person_df.to_csv(f'{output_dir}firms/{output_condition}.csv', 
                       encoding='utf-8', 
                       sep=',', 
                       index=False)


In [ ]:
right_person_df

<a href=#top>先頭に戻る</a>

---


<a id=ipc></a>

### **5.2. IPC**


In [ ]:
schmoch_df = pd.read_csv(f'{ex_dir}35.csv', 
                         encoding='utf-8', 
                         sep=',', 
                         usecols=['Field_number', 'Field_en']
                         ).drop_duplicates()


In [ ]:
# 各期間
classification_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', classification])[['reg_num']].sum().reset_index(drop=False), 
                        c_df.groupby([f'{ar}_{year_style}_period', classification])[[region_corporation]].nunique().reset_index(drop=False), 
                        on=[f'{ar}_{year_style}_period', classification], 
                        how='inner')
classification_df = pd.merge(classification_df, 
                      c_df[[f'{ar}_{year_style}_period', classification, 'ubiquity', 'tci']\
                          +[f'ki_{i}' for i in range(1, 20+1)]]\
                          .drop_duplicates(keep='first'), 
                      on=[f'{ar}_{year_style}_period', classification], 
                      how='inner')
# classification_df['reg_num'] = classification_df['reg_num'].astype(np.int64)
classification_df = pd.merge(classification_df, 
                            schmoch_df.rename(columns={'Field_number':classification}), 
                            on=[classification], 
                            how='inner')\
                            .drop(columns=[classification])\
                            .rename(columns={'Field_en':classification})
display(classification_df)


In [ ]:
classification_df.to_csv(f'{output_dir}technology/{output_condition}.csv', 
                        encoding='utf-8', 
                        sep=',', 
                        index=False)


<a href=#top>先頭に戻る</a>

---


<a id=network></a>

## **5.3. 二部グラフ用**


In [ ]:
# eneos_df = c_df[(c_df[f'{ar}_{year_style}_period']==f'{year_start}-{year_end}')&(c_df[region_corporation].str.contains('ＥＮＥＯＳ'))\
#                 &(c_df['mcp']==1)].copy()#[[region_corporation, 'reg_num', 'schmoch35']].copy()
# eneos_df = pd.merge(eneos_df, 
#                     schmoch_df.rename(columns={'Field_number':'schmoch35'})\
#                               .drop_duplicates(keep='first'), 
#                     on=['schmoch35'], 
#                     how='inner')
# eneos_df[['ubiquity', 'Field_en', 'ki_1']]

In [ ]:
c_df

In [ ]:
graph_df = pd.merge(c_df, schmoch_df, 
                    left_on=classification, right_on='Field_number', how='left')\
                    .drop(columns=['Field_number', classification])\
                    .rename(columns={'Field_en':classification})
# graph_df = graph_df[graph_df['mcp']==1][[f'{ar}_{year_style}', region_corporation, 'ipc_class', 'mcp']]
graph_df

In [ ]:
all_edge_df = graph_df[(graph_df[f'{ar}_{year_style}_period'] == f'{year_start}-{year_end}')&(graph_df['mcp'] == 1)].copy()\
                      [[region_corporation, classification, 'mcp']].rename(columns={'mcp':'Weight'})
all_edge_df['Type'] = 'Undirected'
all_edge_df

all_node_list = list(all_edge_df[region_corporation].unique()) + list(all_edge_df[classification].unique())
all_flag_list = [0] * len(all_edge_df[region_corporation].unique()) + [1] * len(all_edge_df[classification].unique())
all_node_df = pd.DataFrame(all_node_list, columns=['label']).reset_index(drop=False).rename(columns={'index':'node_id'})
all_node_df['projected'] = all_flag_list
all_node_df['node_id'] += 1

all_edge_df = pd.merge(all_edge_df, all_node_df, left_on=region_corporation, right_on='label', how='left').rename(columns={'node_id':'Source'})
all_edge_df = pd.merge(all_edge_df, all_node_df, left_on=classification, right_on='label', how='left').rename(columns={'node_id':'Target'})

all_edge_df = all_edge_df[['Source', 'Target', 'Type', 'Weight']]


In [ ]:
all_node_df.to_csv(f'{output_dir}graph/{output_condition}_node.csv', 
                     encoding='utf-8', 
                     sep=',', 
                     index=False)
all_edge_df.to_csv(f'{output_dir}graph/{output_condition}_edge.csv',
                     encoding='utf-8',
                     sep=',',
                     index=False)


In [ ]:
# graph_df.to_csv(f'{output_dir}graph/{output_condition}.csv', 
#                 encoding='utf-8', 
#                 sep=',', 
#                 index=False)
# graph_df
